In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
print(sys.executable)
import gseapy
from gseapy.plot import barplot, dotplot
warnings.filterwarnings("ignore")
import re
from matplotlib.colors import Normalize

In [ ]:
pre = "MH120"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)
def remove_go(term):
    # Use regular expression to find and replace GO terms, ensuring the string is treated as a raw string
    new_term = re.sub(r'\s*\(GO:\d+\)', '', term)  # Corrected regex finds the GO term pattern and removes it
    return new_term.strip() 

col_human =  "#e35e28"
col_cyno ="#31c7ba"
_,base_table_path_MH115,_,_ = h.return_local_paths(drive = drive,pre = 'MH115',add_path = True)

In [ ]:
df_all_pathways = pd.read_csv(os.path.join(base_table_path_MH115,'MH115_all_df_pathways_.csv'),index_col=0)

In [ ]:
#species = 'cyno'
conditions  = ['timepoints06hr','timepoints24hr']
celltypes_cyno = ['CD4_T', 'CD14_Mono', 'NKProliferating', 'CD8_T', 'B', 'MAIT', 'dnT']
celltypes_human = ['CD4_T', 'CD14_Mono', 'NKProliferating', 'CD8_T', 'B', 'MAIT']
celltypes_both = list(set(celltypes_human)&set(celltypes_cyno))

celltypes_both  = ['NKProliferating', 'B', 'CD4_T','MAIT', 'CD14_Mono','CD8_T']

In [ ]:
heatmap_datas = []
for celltype in celltypes_both:
    pathways_to_show_ct = []
    for condition in conditions:
        df_filt_cyno = df_all_pathways[(df_all_pathways['species'] == 'cyno')&(df_all_pathways['celltype'] == celltype)&(df_all_pathways['comparison.vs.00hr'] == condition)]
        df_filt_human = df_all_pathways[(df_all_pathways['species'] == 'human')&(df_all_pathways['celltype'] == celltype)&(df_all_pathways['comparison.vs.00hr'] == condition)]
        list_coenriched_cyno_human = list(set(list(df_filt_human['Term']))&set(list(df_filt_cyno['Term'])))
        if len(list_coenriched_cyno_human) > 0:
            df_filt_cyno = df_filt_cyno[df_filt_cyno['Term'].isin(list_coenriched_cyno_human)]
            df_filt_human = df_filt_human[df_filt_human['Term'].isin(list_coenriched_cyno_human)]
        if len(df_filt_human) > 0:
            pathways_to_show_ct.append(df_filt_human.head(1).Term.values[0])
        if len(df_filt_cyno) > 0:
            pathways_to_show_ct.append(df_filt_cyno.head(1).Term.values[0])
    
    filtered_df = df_all_pathways[(df_all_pathways['celltype'] == celltype) & df_all_pathways['Term'].isin(pathways_to_show_ct)]
    filtered_df['species_condition'] = filtered_df['species'] + ' - ' + filtered_df['comparison.vs.00hr'].apply(lambda x: x[-4:])
    heatmap_data = filtered_df.pivot_table(index='short_term', columns='species_condition', values='Log Odds Ratio')
    heatmap_data['celltype'] = celltype
    heatmap_datas.append(heatmap_data) 
    
    '''
    plt.figure(figsize=(4, 1))
    norm = Normalize(vmin=0, vmax=6)
    sns.heatmap(heatmap_data, cmap='Reds', annot=True, linewidths=.8,linecolor = 'black',norm=norm)
    plt.title(celltype)
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.show()
    '''

In [ ]:
heatmap_datas

In [ ]:
heatmap_data_all = pd.concat(heatmap_datas)

In [ ]:
heatmap_data_all

In [ ]:
columns_order = ['cyno - 06hr', 'human - 06hr', 'cyno - 24hr', 'human - 24hr','celltype']
heatmap_data_all = heatmap_data_all[columns_order]

In [ ]:
heatmap_data_all.to_csv(os.path.join(base_table_path,pre + '_heatmap_data_all.csv')) 

In [ ]:
plt.figure(figsize=(7, 6))
norm = Normalize(vmin=0, vmax=12)

# Prepare the heatmap without the 'celltype' column
heatmap_ax = sns.heatmap(heatmap_data_all.drop('celltype', axis=1), cmap='Reds', annot=False, linewidths=.8, linecolor='black', norm=norm)

# Set the tick rotation
plt.xticks(rotation=0)
plt.yticks(rotation=0)

# Set y-tick labels to be bold
plt.gca().set_yticklabels(plt.gca().get_yticklabels(), fontweight='bold')

# Create a color bar with a title
cbar = heatmap_ax.collections[0].colorbar
cbar.set_label('log odds ratio',labelpad=10)  
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_pathways.pdf'),bbox_inches='tight')
plt.show()

save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()